<a></a>
# ResNet - Skin Lesions Image Classification with ISIC 2019

Access Drive which contains the Dataset

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Variables Initialization

In [ ]:
# Path where I will be saving the Rotation pretext task model 
filepath_rot = '/content/drive/MyDrive/Tese/fold_1_exp_rot_imagenet'

In [ ]:
# Train fcc
second = False
epochs = 50 
init_lr = 0.0001
#----------------------------------
### Network 
weight_resnet = True #true =imagenet

### Network 
epochs      = 60
batch_size = 32
BATCH_SIZE =32
temperat = 0.1

NUM_TRAIN_SAMPLES = 17731
NUM_VAL_SAMPLES = 7600

In [ ]:
if second == False:
    !unzip /content/drive/MyDrive/Tese/ISIC_2019_pre-processed_cs_final.zip -d /content

<a></a>
## 1.Imports

In [ ]:
import csv
import pandas as pd
import numpy as np
import os,sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from collections import Counter

sys.path.append('/content/drive/MyDrive/Tese/')

from visuals import plot_grouped_2bars
import math
import shutil
from sklearn.utils import shuffle

import glob

import os, os.path
from ast import literal_eval
from make_dir_versao_writedisco import *

import cv2

from PIL import Image


import keras
from tensorflow.keras.layers import Input, Dense, Activation,Flatten, Convolution2D, Conv2D, MaxPooling2D, UpSampling2D, GlobalAveragePooling2D, concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint

from tensorflow.keras import layers, losses

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import preprocessing
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adadelta, RMSprop,SGD,Adam
#from tensorflow.python.keras.preprocessing.image import image_dataset_from_directory

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

from mpl_toolkits.mplot3d import Axes3D

from tensorflow.keras.models import Sequential, save_model, load_model

from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import GridSearchCV , StratifiedShuffleSplit
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report

import seaborn as sns 

from tensorflow.keras.layers import Dense, Dropout, Activation
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

NUM_TRAIN_SAMPLES = 17731
NUM_VAL_SAMPLES = 7600
BATCH_SIZE = 32

<a></a>
## 2. Online Data Augmentation

In [ ]:
from tensorflow.python.keras.applications.resnet import ResNet50, preprocess_input
from keras.preprocessing.image import array_to_img,img_to_array,load_img
from imutils import paths

def parse_images(image_path):
    
    # Load and preprocess the image
    IMG_SHAPE = 224
    img = tf.io.read_file(image_path) # read the raw image
    img = tf.image.decode_jpeg(img, channels=3) # decode the image back to proper format
    img = tf.keras.applications.resnet.preprocess_input(img)
    img = tf.image.resize(img, [IMG_SHAPE, IMG_SHAPE]) # resize the image
    label = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    
    img = tf.image.rot90(img, label) # Rotate 0, 90, 180, 270 degrees
    return img,label

#Create TensorFlow dataset
data_dir1 = '/content/ISIC_2019_pre-processed_cs_final/train'
AUTO = tf.data.AUTOTUNE

image_paths = list(paths.list_images(data_dir1))
list_ds = tf.data.Dataset.from_tensor_slices((image_paths))

print('\n\n Loading dataset ')
train_ds = (
    list_ds
    .map(parse_images, num_parallel_calls=AUTO)
    .shuffle(1024)
    # .map(augment, num_parallel_calls=AUTO) # augmentation call
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(AUTO)
)

#Create TensorFlow dataset
data_dir1 = '/content/ISIC_2019_pre-processed_cs_final/valid'
AUTO = tf.data.AUTOTUNE

image_paths = list(paths.list_images(data_dir1))
list_ds = tf.data.Dataset.from_tensor_slices((image_paths))

print('\n\n Loading dataset ')
valid_ds = (
    list_ds
    .map(parse_images, num_parallel_calls=AUTO)
    .shuffle(1024)
    # .map(augment, num_parallel_calls=AUTO) # augmentation call
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(AUTO)
)
print((train_ds))
print((valid_ds))




<a></a>
## 3.Build ResNet Architecture

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50
if weight_resnet == True:
  MODEL = ResNet50(include_top=False, weights= 'imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None)
  # MODEL.summary()
  print('imagenet weighths loaded')
elif weight_resnet == False:
  MODEL = ResNet50(include_top=False, weights= None, input_tensor=None, input_shape=(224,224,3), pooling=None)
  # MODEL.summary()

94773248/94765736 [==============================] - 0s 0us/step
imagenet weighths loaded


In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D

model = tf.keras.Sequential(MODEL)
# Flatten the output layer to 1 dimension
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(4))

optimizer = tf.keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, decay=0.0)
model.compile(optimizer=optimizer,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

saveWeights = ModelCheckpoint(filepath_rot, save_best_only=True, monitor='val_loss', mode='min')
learningRate = ReduceLROnPlateau(monitor='val_loss', factor=0.75, patience=5, verbose=1,  mode='min')


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


<a></a>
## 3. Fit

In [ ]:

history = model.fit(train_ds,epochs=epochs,
          validation_data=(valid_ds),
          shuffle = True,
          # workers = 4, 
          # callbacks =[LearningRateScheduler(lr_scheduler, verbose=1),saveWeights,learningRate, tensBoard],
          callbacks =[saveWeights,learningRate],
          #validation_steps = NUM_VAL_SAMPLES // BATCH_SIZE,
          # class_weight = class_weights
          )

In [ ]:
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()